Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import azureml.core
from azureml.core import Workspace
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import os

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
project_folder=os.getcwd()
data_folder='trainDataset'
script_folder = os.path.join(os.getcwd(), "scripts")


### Connect to workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `ws`.

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

### Create experiment

Create an experiment to track the runs in your workspace. A workspace can have muliple experiments. 

In [ ]:
experiment_name = 'ImageClassifier'

from azureml.core import Experiment
experiment = Experiment(workspace=ws, name=experiment_name)

### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

In [ ]:
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           min_nodes=0,
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

### Visualize Trainig Data

In [ ]:
with open(os.path.join('trainDataset', "x_train.tsv"),"rb") as f:
     x_train=pickle.load(f)
with open(os.path.join('trainDataset', "y_train.tsv"),"rb") as f:
     y_train=pickle.load(f)
with open(os.path.join('trainDataset', "x_val.tsv"),"rb") as f:
     x_val=pickle.load(f)
with open(os.path.join('trainDataset', "y_val.tsv"),"rb") as f:
     y_val=pickle.load(f)
with open(os.path.join('testDataset', "x_test.tsv"),"rb") as f:
     x_test=pickle.load(f)
with open(os.path.join('testDataset', "y_test.tsv"),"rb") as f:
     y_test=pickle.load(f)
with open(os.path.join('trainDataset', "encoder"),"rb") as f:
     encoder=pickle.load(f)

### Visualize the data

In [ ]:
plt.imshow(x_train[1], cmap=plt.cm.Greys)

### Upload data to the cloud

In [ ]:
ds.upload(src_dir=data_folder, target_path='trainDataset', overwrite=True, show_progress=False)
ds.upload(src_dir=data_folder, target_path='testDataset', overwrite=True, show_progress=False)
print('ready!')

### Preparation for the model

In [ ]:
ntrain=len(x_train)
nval=len(x_val)
batch_size=32

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
script_folder

In [ ]:
%%writefile $script_folder/train_onnx.py
import argparse
import os
import numpy as np
import pickle
from azureml.core.run import Run
from keras import layers, models, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
##################################################################################    <-------     ONNX
import onnx                                                                          #<-------     ONNX
import winmltools                                                                    #<-------     ONNX

run = Run.get_context()

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--batch_size', type=str, dest='batch_size', help='batch_size')

#parser.add_argument('--output', type=float, dest='output', default=0.01, help='regularization rate')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)
batch_size = int(args.batch_size)
print('Batch size:', batch_size)


with open(os.path.join(data_folder, 'x_train.tsv'),"rb") as f:
    x_train = pickle.load(f)
with open(os.path.join(data_folder,'y_train.tsv'),"rb") as f:
    y_train = pickle.load(f)
with open(os.path.join(data_folder,'x_val.tsv'),"rb") as f:
    x_val = pickle.load(f)
with open(os.path.join(data_folder,'y_val.tsv'),"rb") as f:
    y_val = pickle.load(f)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, sep = '\n')

ntrain=len(x_train)
nval=len(x_val)


###################  modeling part

model=models.Sequential()
model.add(layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(8,activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-3), metrics=['accuracy'])

trainDataGen= ImageDataGenerator(
                    rescale=1.,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2
                    )
valDataGen=ImageDataGenerator(rescale=1.)

trainGenerator=trainDataGen.flow(x_train,y_train,batch_size = batch_size)
valGenerator=valDataGen.flow(x_val,y_val,batch_size = batch_size)

early_stopping = EarlyStopping(monitor='val_loss', patience=2)
history=model.fit_generator(trainGenerator,
                            steps_per_epoch = ntrain//batch_size,
                            epochs = 16,
                            validation_data = valGenerator,
                            validation_steps = nval//batch_size,
                            callbacks=[early_stopping],
                            workers=3
                           )

run.log_list('val_accuracy', history.history['val_acc'], description='Validation accuracy')
run.log_list('val_loss', history.history['val_loss'], description='validation loss')

# for now I am not using the output argument from the args
output_dir = './outputs/model'
os.makedirs(output_dir, exist_ok=True)
model_path = os.path.join(output_dir, 'onnxmodel.onnx')

##################################################################################    <-------     ONNX
TARGET_OPSET = 8 #8 for ONNX 1.3.                                                    #<------- 
convert_model = winmltools.convert_keras(model,TARGET_OPSET)                         #<-------  
winmltools.save_model(convert_model, model_path)                                     #<-------  

'''with open('./outputs/model/trainHistoryDict', 'wb') as f:
        pickle.dump(history.history, f)'''

### Create an estimator

An estimator object is used to submit the run. Azure Machine Learning has pre-configured estimators for common machine learning frameworks, as well as generic Estimator. Create SKLearn estimator for scikit-learn model, by specifying

* The name of the estimator object, `est`
* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will use the AmlCompute you created
* The training script name, train.py
* Parameters required from the training script 


In [ ]:
from azureml.train.dnn import TensorFlow
from azureml.core.runconfig import MpiConfiguration

script_params = {
    '--data-folder': ds.path('trainDataset').as_mount(),
    '--batch_size': 16
    
}

##################################################################################    <-------     ONNX
est = TensorFlow(source_directory=script_folder,
                 entry_script='train_onnx.py',
                 script_params=script_params,
                 compute_target=compute_target,
                 pip_packages=['keras', 'winmltools','onnx'], #<-------     
                 node_count=3,
                 process_count_per_node=1,
                 distributed_training=MpiConfiguration(),
                 framework_version="1.13",                    #<-------     
                 use_gpu=True)

This is what the mounting point looks like:

### Submit the job to the cluster

Run the experiment by submitting the estimator object. And you can navigate to Azure portal to monitor the run.

In [ ]:
%%writefile $project_folder/.amlignore
data
model
sourceData
Training
Testing
trainDataset
testDataset
dataLH
model_keras.h5
model_weights.h5
model_wieghts.h5
ARCH

In [ ]:
run = experiment.submit(config=est)
run

## Monitor a remote run


In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()


### Download the model (optional)

In [ ]:
##################################################################################    <-------     ONYX
model_path = os.path.join('outputs','model', 'onnxmodel.onnx')
run.download_file(model_path, output_file_path=model_path)   #############################################?

In [ ]:
model = run.register_model(model_name='onnxmodel', model_path=model_path)
print(model.name, model.id, model.version, sep = '\t')

### Or any model from the workspace (optional)

In [ ]:
models = ws.models
for name, m in models.items():
    print("Name:", name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

In [ ]:
from azureml.core.model import Model
import os

model = Model(workspace=ws, name="onyxmodel", version=10)
model.download(target_dir=os.getcwd(),exist_ok=True)
model

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/img-classification-part1-training.png)

# Deploy

In [ ]:
%%writefile score.py

import json
import numpy as np
from azureml.core.model import Model
import onnxruntime as rt

def init():
    global session
    model_path = Model.get_model_path(model_name='onnxmodel')
    session = rt.InferenceSession(model_path)


def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    data = data.astype(np.float32)
    input_name = session.get_inputs()[0].name
    
    # make prediction
    res = session.run(None, {input_name: data})
    
    prob = res[0]

    return prob.tolist()

### Creating container image

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 
##################################################################################          <-------     ONYX
myenv = CondaDependencies.create(pip_packages=["numpy","onnxruntime","azureml-defaults"]   #<-------     ONYX
                                )
myenv.add_conda_package('tensorflow')
myenv.add_conda_package('keras')
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime="python",
                                   entry_script="score.py",
                                   conda_file="myenv.yml")

In [ ]:
from azureml.core.webservice import  AciWebservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration()
service = Model.deploy(ws, "myonnxservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

===========================================================













==============================================================

### Test

In [ ]:
import json

with open(os.path.join('testDataset', "x_test.tsv"),"rb") as f:
     x_test=pickle.load(f)
with open(os.path.join('testDataset', "y_test.tsv"),"rb") as f:
     y_test=pickle.load(f)
with open(os.path.join('trainDataset', "encoder"),"rb") as f:
     encoder=pickle.load(f)

idx=12
test_sample_x=x_test[idx]
test_sample_y=y_test[idx]

testx=np.expand_dims(test_sample_x, axis=0)
test_json=json.dumps({'data':testx.tolist()})

In [ ]:
predicted = service.run(input_data = test_json)
result=pd.DataFrame(np.around(predicted[0],3))

In [ ]:
result['MaxVal']=''
result['MaxVal'][np.argmax(predicted)]= ' *'
result

In [ ]:
predLab=np.argmax(predicted)
predictedLabel= encoder.inverse_transform([predLab])

In [ ]:
plt.figure()
labelsValid=np.core.defchararray.add(np.core.defchararray.add(test_sample_y,' / '),predictedLabel[0])

title_obj=plt.title(labelsValid, fontsize=12)
if (predictedLabel[0]!=test_sample_y):
        plt.setp(title_obj, color='r') 
plt.imshow(test_sample_x)